## __Confined_01__

In [1]:
from pathlib import Path
from tempfile import TemporaryDirectory
import numpy as np
import flopy

In [3]:
temp_dir = TemporaryDirectory()
workspace = Path(temp_dir.name)
name = "tutorial01_mf"     
mf = flopy.modflow.Modflow(name, exe_name="mf2005", model_ws=workspace)                    # *.nam        # MF2005

In [4]:
Lx, Ly           = 1000.0, 1000.0
ztop, zbot       = 0.0, -50.0
nper, nlay, nrow, ncol = 1, 1, 10, 20                   # nper = 1  (add)
delr, delc       = Lx / ncol, Ly / nrow
delv             = (ztop - zbot) / nlay
botm             = np.linspace(ztop, zbot, nlay + 1)   # xoffset, yoffset = 554400., 389200.0   # epsg = 3070      # *.dis   # modelgrid = ?
dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc, top=ztop, botm=botm[1:])

In [5]:
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:, :, 0] = -1
ibound[:, :, -1] = -1
strt = np.ones((nlay, nrow, ncol), dtype=np.float32)
strt[:, :, 0] = 10.0
strt[:, :, -1] = 0.0
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)                               # *.bas _______________
mf.write_input()

In [6]:
lpf = flopy.modflow.ModflowLpf(mf, hk=10.0, vka=10.0, ipakcb=53)
spd = {(0, 0): ["print head", "print budget", "save head", "save budget"]}        # stress period and time step.
oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd, compact=True)

In [7]:
pcg = flopy.modflow.ModflowPcg(mf)                                                # solver Precond. Conj. Grad.
mf.write_input()

In [8]:
success, buff = mf.run_model()   # model object ___mf____ has an attached method that will run the mode ____mf2005
assert success, "nope."

FloPy is using the following executable to run the model: E:\15_REPOS\00_BETAMI\02_floPy__________________\05_mf______________________________________ok\01__mf_tutorial01___Confined______ok\mf2005.exe

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: tutorial01_mf.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2025/02/03 11:45:34

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2025/02/03 11:45:34
 Elapsed run time:  0.032 Seconds

  Normal termination of simulation


## ___Results_01__

In [ ]:
import matplotlib.pyplot as plt
import flopy.utils.binaryfile as bf

In [ ]:
hds = bf.HeadFile(Path(workspace) / f"{name}.hds")           # __________ Flopy binary utility to read the heads __________ 
head = hds.get_data(totim=1.0)
times = hds.get_times()
head = hds.get_data(totim=times[-1])

In [ ]:
extent = (delr / 2.0, Lx - delr / 2.0, Ly - delc / 2.0, delc / 2.0)      # delr / 2.0  = 
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
ax.contour(head[0, :, :], levels=np.arange(1, 10, 1), extent=extent)      # plt.tight_layout()

## ___Results_02__

In [ ]:
cbb = bf.CellBudgetFile(Path(workspace) / f"{name}.cbc")                 # ____ CHD ______ ?
kstpkper_list = cbb.get_kstpkper()
#frf = cbb.get_data(text="FLOW RIGHT FACE", totim=times[-1])[0]
#fff = cbb.get_data(text="FLOW FRONT FACE", totim=times[-1])[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge((frf, fff, None), mf, head)    # _____frf  _____ fff 

In [ ]:
fig = plt.figure(figsize=(3, 3))
ax = fig.add_subplot(1, 1, 1, aspect="equal")
modelmap = flopy.plot.PlotMapView(model=mf, layer=0, ax=ax)          # mf 
qm = modelmap.plot_ibound()
lc = modelmap.plot_grid()
cs = modelmap.contour_array(head, levels=np.linspace(0, 10, 11))     # countor
#quiver = modelmap.plot_vector(qx, qy)

In [ ]:
try:
    temp_dir.cleanup()
except:
    # prevent windows permission error
    pass